In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.lines import Line2D

# Carga de tablas 

In [96]:
# Load the XLSX files into Pandas dataframes
Inventario = pd.read_excel('Inventarios/19-04.xlsx', skiprows=2)
Ventas = pd.read_excel('Ventas/Ventas.xlsx', skiprows=2)
Utilidad = pd.read_excel('Ventas/Utilidad.xlsx', skiprows=2)
Cartera = pd.read_excel('Cartera/19-04.xlsx', skiprows=2)
Egresos = pd.read_excel('Egresos/Egresos.xlsx', skiprows=2)
Egresos_ = pd.read_excel('Egresos/Egresos_.xlsx', skiprows=2)
Pendiente = pd.read_excel('Suministro/Pendiente.xlsx', skiprows=2)
IQChemicals = pd.read_excel('IQChemicals/Otros.xlsx', skiprows=2)
IQChemicals_mp = pd.read_excel('IQChemicals/Materia_prima.xlsx', skiprows=2)
CxP = pd.read_excel('CxP/CxP.xlsx', skiprows=2)

# Tabla auxiliar para traducir unidades en costo
Pres_a_vol = pd.read_excel('Categorias/Presentacion_a_volumen.xlsx')
Item_a_producto = pd.read_excel('Categorias/Item_a_producto.xlsx')
Costos_unitarios = pd.read_excel('Costo_unitario/Costo.xlsx',skiprows=2)
Producto = pd.read_excel('Categorias/Producto.xlsx')


#La hoja de inventario IqChemicals, tiene dos tablas. Para el análisis, se consolidan en una:

In [97]:
IQChemicals_mp_otros = pd.concat([IQChemicals,IQChemicals_mp])

Cartera, se desglosa en Vencida y corriente

In [98]:
Cartera['ESTADO'] = np.where(Cartera['ANTIGUEDAD'] == '  VENCIDA', 'VENCIDA', 'CORRIENTE')

Cartera corriente

In [99]:
Cartera_corriente = Cartera[Cartera['ESTADO'] == 'CORRIENTE']
Cartera_vencida = Cartera[Cartera['ESTADO'] == 'VENCIDA']

In [100]:
#Inventario, incluir columna de ABC

Item_a_ABC = Item_a_producto.groupby(['Item','ABC']).size().reset_index()
Inventario = Inventario.rename(columns = {'Número de artículo':'Item'})
Inventario = Inventario.merge(Item_a_ABC, how = 'left', on = 'Item')

In [101]:
#Ventas, incluir columna de categoría

Producto = Producto.groupby(['Producto_Ranking','Respaldo categorías']).size().reset_index()

Producto['Producto_Ranking'] = Producto['Producto_Ranking'].str.upper().str[3:]

Ventas['Producto_Ranking'] = Ventas['Producto_Ranking'].str[3:]
Utilidad['Producto_Ranking'] = Utilidad['Producto_Ranking'].str[3:]


Limpieza de homógos
JUNIPERUS no es igual que JUNÍPERUS... LPG 456 REACTMAXX, no es igual que LPG 456 -REACTMAXX...
La siguiente sección incluye algunos homólogos para que todos las filas crucen:

In [102]:
# Listado de productos que no cruzaron originalmente (Limpieza de homólogos)
Homologos_adicionales = pd.DataFrame({
    'Producto_Ranking': ['JUNIPERUS STRESSOFF', 'SPROUD MAX', 'LPG 456 - REACTMAXX', 'AGROPLEXBMO STRESSOFF', 
                'KIT-CALIDAD-IQC', 'KIT-ESTRUCTURAL-IQC', 'OTROS'],
    'Respaldo categorías': ['Bioestimulante', 'Cito', 'Cito', 'Bioestimulante', 
                 'Bioestimulante', 'Cito', 'Bioestimulante']
})

# append the corrected mapping to the producto dataframe
Producto = pd.concat([Producto,Homologos_adicionales], ignore_index=True)



In [103]:
Ventas = Ventas.merge(Producto, on = 'Producto_Ranking', how = 'left')
Utilidad = Utilidad.merge(Producto, on = 'Producto_Ranking', how = 'left')

In [104]:
#Pendiente, incluir columna de categoría
Item_a_categoría = Item_a_producto.groupby(['Item','Categoría']).size().reset_index()
Pendiente = Pendiente.rename(columns = {'ITEMNMBR':'Item'})
Pendiente = Pendiente.merge(Item_a_categoría, how = 'left', on = 'Item')

In [105]:
#IQChemicals_homologación de categorías

# Create a new column with updated values
IQChemicals_mp_otros['ALMACEN_'] = np.where(IQChemicals_mp_otros['ALMACEN'].isin(['ALMACEN DE PRODUCTO TERMINADO CHEMICALS', 'PRODUCTO TERMINADO TLAXCALA']),
                                              'PRODUCTO TERMINADO',
                                              IQChemicals_mp_otros['ALMACEN'])

# Indice de tablas

In [106]:
#df - Ventas
#df1 - Utilidad
#df2 - Inventario
#df3 - Cartera_vencida
#df4 - Egresos
#df5 - Pendiente
#df6 - Costo
#df7 - Inventario IQC
#df8 - Cartera_corriente
#df9 - CxP

# Homologación de nombres de columnas

In [107]:
#Ventas 2023, valores superiores a cero, renombrar columnas
Ventas = Ventas.rename(columns={
       'Empresa Jerarquía - Zona Ranking_':'Zona',
       'Empresa Jerarquía - RM/ REIQ_Ranking':'RM',
       'Empresa Jerarquía - Cultivo_ranking':'Cultivo',
       'ID_PRODUCTO':'Item', 
       "'Base'[Mes]'":"Mes",
       'Ver selección':'Ventas',
       'Respaldo categorías':'Nivel 2'
                        }
               )
Ventas = Ventas[Ventas['Ventas'] > 0]

In [108]:
#Utilidad 2023, renombrar columnas
Utilidad = Utilidad.rename(columns={
       'Empresa Jerarquía - Zona Ranking_':'Zona',
       'Empresa Jerarquía - RM/ REIQ_Ranking':'RM',
       'Empresa Jerarquía - Cultivo_ranking':'Cultivo',
       'ID_PRODUCTO':'Item', 
       "'Base'[Mes]'":"Mes",
       'Ver selección':'Utilidad',
       'Respaldo categorías':'Nivel 2'
                        }
               )
#Filtros de Utilidad 2023, valores superiores a cero
Utilidad = Utilidad[Utilidad['Utilidad'] > 0]

In [109]:

#Inventario, renombrar columnas
Inventario = Inventario.rename(columns = {
    'EMPRESA':'Empresa',
    'VALOR INVENTARIO':'Inventario_valor',
    'Número de artículo':'Item',
    'ABC':'Nivel 2'
    
                                        }
                              )
#Filtros de Inventario, valores superiores a uno, 

Inventario = Inventario[Inventario['Inventario_valor'] > 0]

In [110]:
#Cartera vencida, renombrar columnas
Cartera_vencida = Cartera_vencida.rename(columns = {
    "'VW_CxC_COM_V4'[EMPRESA]":"Empresa",
    "Sum of TOTAL":"Cartera_vencida",
    "ANTIGUEDAD":"Nivel 2"
                                        }
                              )
#Filtros de Cartera, valores superiores a uno, 

Cartera_vencida = Cartera_vencida[Cartera_vencida['Cartera_vencida'] > 0]

In [111]:
#Cartera_corriente, renombrar columnas
Cartera_corriente = Cartera_corriente.rename(columns = {
    "'VW_CxC_COM_V4'[EMPRESA]":"Empresa",
    "Sum of TOTAL":"Cartera_corriente",
    "ANTIGUEDAD":"Nivel 2"
                                        }
                              )
#Filtros de Cartera_corriente, valores superiores a uno, 

Cartera_corriente = Cartera_corriente[Cartera_corriente['Cartera_corriente'] > 0]

In [112]:
#Egresos, renombrar columnas
Egresos_ = Egresos_.rename(columns = {
    "NIVEL4":"Empresa",
    "Sum of MONTO":"Egresos_monto",
    "NIVEL2":"Nivel 2"
                                        }
                              )
#Filtros de Egresos, valores superiores a uno, 

Egresos_ = Egresos_[Egresos_['Egresos_monto'] > 0]
Egresos_ = Egresos_[Egresos_['FUENTE'] == "REAL"]

In [113]:
#Pendiente, renombrar columnas
Pendiente = Pendiente.rename(columns = {
    "EMPRESA":"Empresa",
    "Sum of PENDIENTE":"Unidades",
    "PRESENTACION":"Presentación",
    "Categoría":"Nivel 2"
                                        }
                              )
#Filtros de Egresos, valores superiores a uno, 

Pendiente = Pendiente[Pendiente['Unidades'] > 0]
Formulaciones_a_excluir = ['AGE-21', 'AABC-22', 'ARP4-20', 'AC GL-22']
Pendiente = Pendiente[~Pendiente['FORMULACION'].isin(Formulaciones_a_excluir)]

In [114]:
#Pres_a_vol, NO requiere renombrar columnas
#Pres_a_vol.columns 

Listado de cols:
'Presentación', 'Vol aux'

In [115]:
#Costos_unitarios, renombrar columnas
Costos_unitarios = Costos_unitarios.rename(columns = {
    "PRESENTACION":"Presentación"
                                        }
                              )
#Filtros de Costos_unitarios, valores superiores a uno, 

Costos_unitarios = Costos_unitarios[Costos_unitarios['Sum of K/L'] > 0]

In [116]:
#Costos_unitarios, renombrar columnas
IQChemicals_mp_otros = IQChemicals_mp_otros.rename(columns = {
    "EMPRESA":"Empresa",
    "VALOR INVENTARIO":"Inventario_IQC_Valor",
    "ALMACEN_":"Nivel 2"
                                        }
                              )
#Filtros de Costos_unitarios, valores superiores a uno, 

IQChemicals_mp_otros = IQChemicals_mp_otros[IQChemicals_mp_otros['Inventario_IQC_Valor'] > 0]

In [117]:
CxP.columns

Index(['GPO', 'EMPRESA', 'RUBRO', 'PROVEEDOR', 'TOTAL', 'TIPOS'], dtype='object')

In [118]:
#CxP, renombrar columnas
CxP = CxP.rename(columns = {
    "EMPRESA":"Empresa",
    "TOTAL":"CxP",
    "RUBRO":"Nivel 2"
                                        }
                              )
#Filtros de Costos_unitarios, valores superiores a uno, 

CxP = CxP[CxP['CxP'] > 0]

# Tabla auxiliar para pasar de unidades a costo

In [119]:
#Costo por formulación y presentación
Costos_unitarios = Costos_unitarios.groupby(['FORMULACION','Presentación'])['COSTOUNITREAL.'].mean().reset_index()

In [120]:
# Costos NO incluidos en tablero.
new_data = pd.DataFrame({
    'FORMULACION': ['APLG-21', 'BRXCO-21', 'AV5G-22', 'AV5G-22', 'BTR-22'],
    'Presentación': ['1X200', '12X1', '1X20', '4X5', '4X5'],
    'COSTOUNITREAL.': [20, 70, 80, 80, 200]
})

# Se consolida tabla para que no haya valores en blanco al cruzar con Pendiente
Costos_unitarios = pd.concat([Costos_unitarios, new_data], ignore_index=True)


In [121]:
#Factor de volumenn por presentación
Pres_a_vol_ = Pres_a_vol.groupby(['Presentación'])['Vol aux'].mean().reset_index()

In [122]:
#Unidades a volumen
Pendiente = Pendiente.merge(Pres_a_vol_, on = 'Presentación', how = 'left')
Pendiente['Volumen'] = Pendiente['Vol aux']*Pendiente['Unidades']
# Volumen a costo
Pendiente = Pendiente.merge(Costos_unitarios, on = ['FORMULACION', 'Presentación'], how = 'left')
Pendiente['Pendiente_valor'] = Pendiente['Volumen']*Pendiente['COSTOUNITREAL.']

# Agrupar componentes financieros por empresa

In [123]:
# group Ventas por empresa
df = Ventas.groupby(['Empresa','Nivel 2'])['Ventas'].sum().reset_index()
df['Fuente'] = "Ventas"

In [124]:
# group Utilidad por empresa
df1 = Utilidad.groupby(['Empresa','Nivel 2'])['Utilidad'].sum().reset_index()
df1['Fuente'] = "Utilidad"

In [125]:
# group Inventario por empresa
df2 = Inventario.groupby(['Empresa','Nivel 2'])['Inventario_valor'].sum().reset_index()
df2['Fuente'] = "Inventario_valor"
df2 = df2.rename(columns = {'Inventario_valor':'Valor'})

In [126]:
Cartera_vencida.head()

,GPO,Empresa,ZONA,CLIENTE,CATEGORIA,Cartera_vencida,Nivel 2,ESTADO
6,GRUPO,INT,"Ecu,Col",AGRICOLAS ANTIOQUIA SAS,AdStrong,2.044621e+06,VENCIDA,VENCIDA
8,GRUPO,CAC,Zona 2,AGRICOLA PISCIS SA,AdStrong,1.764476e+06,VENCIDA,VENCIDA
9,GRUPO,IQCHEM,REIQ,CORPORATIVO TXAO,ReactMaxx,1.320918e+06,VENCIDA,VENCIDA
11,GRUPO,IQCHEM,REIQ,CORPORATIVO TXAO,StressOff,1.305532e+06,VENCIDA,VENCIDA
18,GRUPO,CAC,Zona 2,AGROASA,ReactMaxx,1.113007e+06,VENCIDA,VENCIDA


In [127]:
# group Cartera por empresa
df3 = Cartera_vencida.groupby(['Empresa','Nivel 2'])['Cartera_vencida'].sum().reset_index()
df3['Fuente'] = "Cartera_vencida"
df3 = df3.rename(columns = {'Cartera_vencida':'Valor'})

In [128]:
# group Egresos por empresa
df4 = Egresos_.groupby(['Empresa','Nivel 2'])['Egresos_monto'].sum().reset_index()
df4['Fuente'] = "Egresos_monto"
df4 = df4.rename(columns = {'Egresos_monto':'Valor'})

In [129]:
# group Pendiente por empresa
df5 = Pendiente.groupby(['Empresa','Nivel 2'])['Pendiente_valor'].sum().reset_index()
df5['Fuente'] = "Pendiente_valor"
df5 = df5.rename(columns = {'Pendiente_valor':'Valor'})

# Costo

In [130]:
# Combinar Utilidad y Ventas
df_merged = pd.merge(df, df1, on=['Empresa', 'Nivel 2'], how='left')


# Calcular Costo como diferencia de Ventas e Utilidad
df_merged['Costo'] = df_merged['Ventas'] - df_merged['Utilidad']

# Agrupar Costo por Empresa
df6 = df_merged.groupby(['Empresa','Nivel 2'])['Costo'].sum().reset_index()
df6['Fuente'] = "Costo"
df6 = df6.rename(columns = {'Costo':'Valor'})

Agregar columna Fuente en Ventas e Utilidad
Se agregó después porque esas columnas iban a utilizarse para la tabla de costos

In [131]:
df_ = df.rename(columns = {'Ventas':'Valor'})
df1_ = df1.rename(columns = {'Utilidad':'Valor'})

# Inventario IQC valor

In [132]:
# group Inventario IQC por empresa
df7 = IQChemicals_mp_otros.groupby(['Empresa','Nivel 2'])['Inventario_IQC_Valor'].sum().reset_index()
df7['Fuente'] = "Inventario_IQC_Valor"
df7 = df7.rename(columns = {'Inventario_IQC_Valor':'Valor'})

In [133]:
# group Cartera corriente
df8 = Cartera_corriente.groupby(['Empresa','Nivel 2'])['Cartera_corriente'].sum().reset_index()
df8['Fuente'] = "Cartera_corriente"
df8 = df8.rename(columns = {'Cartera_corriente':'Valor'})


In [134]:
# group CxP
df9 = CxP.groupby(['Empresa','Nivel 2'])['CxP'].sum().reset_index()
df9['Fuente'] = "CxP"
df9 = df9.rename(columns = {'CxP':'Valor'})

# Componentes financieros consolidados

In [135]:
dfs = [df_, df1_, df2, df3, df4, df5, df6, df7, df8, df9]
cols = ['Empresa', 'Nivel 2', 'Fuente', 'Valor']
df_merged = pd.concat(dfs, axis=0)



# Categorías de componentes

In [136]:
# Define the conditions and categories
conditions = [df_merged['Fuente'].isin(['Ventas', 'Utilidad']),
    df_merged['Fuente'].isin(['Egresos_monto', 'Costo', 'CxP']),
    df_merged['Fuente'].isin(['Cartera_vencida', 'Cartera_corriente','Inventario_valor', 'Pendiente_valor','Inventario_IQC_Valor'])
]

categories = ['01 Ingresos','02 Egresos','03 Inventario y Cartera']

# Apply the conditions and categories to create a new "Category" column
df_merged['Categoria'] = np.select(conditions, categories)

In [137]:
df_merged.to_csv('Componentes_financieros_nivel_2.csv')